# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('joins').getOrCreate()
spark

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os

path= 'Datasets/uw-madison-courses/'

df_list = []
for filename in os.listdir(path):
    if filename.endswith('.csv'):
        filename_list = filename.split('.')
        df_name = filename_list[0]
        df= spark.read.csv(path+filename,inferSchema=True, header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')

print('Full list of dfs:')
print(df_list)

Full list of dfs:
['subjects', 'subject_memberships', 'rooms', 'schedules', 'sections', 'courses', 'course_offerings', 'instructors', 'teachings', 'grade_distributions']


Now check the contents of a few of the dataframses that were read in above.

In [3]:
subjects.show(5)

+----+--------------------+------------+
|code|                name|abbreviation|
+----+--------------------+------------+
| 908|        Soil Science|    SOIL SCI|
| 350|             ENGLISH|     ENGLISH|
| 351|English as a Seco...|         ESL|
| 230|Human Development...|        HDFS|
| 352|             English|        ENGL|
+----+--------------------+------------+
only showing top 5 rows



In [4]:
schedules.show(5)

+--------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
|                uuid|start_time|end_time|  mon| tues|  wed|thurs|  fri|  sat|  sun|
+--------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
|f2d66a4d-0c08-3b4...|        -1|      -1|false|false|false|false|false|false|false|
|875542a2-f786-34d...|       530|     580| true|false| true|false|false|false|false|
|f41f1e4d-cb4f-3de...|       660|     710|false| true|false| true|false|false|false|
|46da55a4-17f7-31a...|       800|     850|false| true|false| true|false|false|false|
|8c7cd81e-4f81-357...|       725|     775|false| true|false| true|false|false|false|
+--------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows



In [5]:
rooms.show(5)

+--------------------+-------------+---------+
|                uuid|facility_code|room_code|
+--------------------+-------------+---------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|
|2cc50da3-ef0e-357...|         0032|     0249|
|ebbf62b4-2ac3-356...|         0032|     B101|
|ed828265-475b-31b...|         0032|     0549|
|b277dc8e-bab1-3a1...|         0032|     0349|
+--------------------+-------------+---------+
only showing top 5 rows



## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [78]:
course_offerings.show(5)

+--------------------+--------------------+---------+--------------------+
|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
|f718e6cd-33f0-3c1...|a3e3e1c3-543d-3bb...|     1082|Cooperative Educa...|
|ea3b717c-d66b-30d...|a3e3e1c3-543d-3bb...|     1172|Cooperative Educa...|
|075da420-5f49-3dd...|a3e3e1c3-543d-3bb...|     1114|Cooperative Educa...|
|2b4e216d-a728-371...|a3e3e1c3-543d-3bb...|     1104|Cooperative Educa...|
+--------------------+--------------------+---------+--------------------+
only showing top 5 rows



In [79]:
rooms.show(5)

+--------------------+-------------+---------+
|                uuid|facility_code|room_code|
+--------------------+-------------+---------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|
|2cc50da3-ef0e-357...|         0032|     0249|
|ebbf62b4-2ac3-356...|         0032|     B101|
|ed828265-475b-31b...|         0032|     0549|
|b277dc8e-bab1-3a1...|         0032|     0349|
+--------------------+-------------+---------+
only showing top 5 rows



In [80]:
step1 = rooms.join(sections, sections.room_uuid == rooms.uuid, how='left').select('room_uuid', 'facility_code', 'room_code', 'course_offering_uuid')

In [81]:
step2 = step1.join(course_offerings, step1.course_offering_uuid== course_offerings.uuid, how='left').select('uuid','room_uuid', 'facility_code', 'room_code', 'term_code', 'name')

In [82]:
step2.show()

+--------------------+--------------------+-------------+---------+---------+--------------------+
|                uuid|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+--------------------+-------------+---------+---------+--------------------+
|611e5499-724c-346...|0feb8e9a-88b8-3d8...|         0140|     4349|     1182|International Bus...|
|43688423-905c-345...|0feb8e9a-88b8-3d8...|         0140|     4349|     1174|Intro Managerial ...|
|43688423-905c-345...|0feb8e9a-88b8-3d8...|         0140|     4349|     1174|Intro Managerial ...|
|871476fd-340f-303...|0feb8e9a-88b8-3d8...|         0140|     4349|     1174|        Business Law|
|8a58936e-d4ad-3c6...|0feb8e9a-88b8-3d8...|         0140|     4349|     1182|        Business Law|
|8a58936e-d4ad-3c6...|0feb8e9a-88b8-3d8...|         0140|     4349|     1182|        Business Law|
|8a58936e-d4ad-3c6...|0feb8e9a-88b8-3d8...|         0140|     4349|     1182|        Business Law|
|8a58936e-

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [38]:
step2[(step2['facility_code']== '0469')].show()

+--------------------+-------------+---------+---------+--------------------+
|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     2441|     1152|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1092|  Fundamentals-Flute|
|6af80b0b-b3e3-370...|         0469|     4411|     1172|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1162|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1132|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1072|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1142|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1112|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1102|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1082|  Fundam

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [ ]:
subjects: code

subject_memberships: subject_code, course_offering_uuid

In [54]:
subjects.show()

+----+--------------------+------------+
|code|                name|abbreviation|
+----+--------------------+------------+
| 908|        Soil Science|    SOIL SCI|
| 350|             ENGLISH|     ENGLISH|
| 351|English as a Seco...|         ESL|
| 230|Human Development...|        HDFS|
| 352|             English|        ENGL|
| 231|    General Business|     GEN BUS|
| 232|Accounting and In...|    ACCT I S|
| 112|Biological System...|         BSE|
| 233|Finance, Investme...|     FINANCE|
| 476|        Horticulture|        HORT|
| 355|          Entomology|       ENTOM|
| 234| Information Systems|    INFO SYS|
| 235|International Bus...|    INTL BUS|
| 236|Management and Hu...|       M H R|
| 237|           Marketing|    MARKETNG|
| 359|      Design Studies|          DS|
| 238|Operations and Te...|         OTM|
| 239|Real Estate and U...|    REAL EST|
| 912|Spanish (Spanish ...|     SPANISH|
| 480|      Human Oncology|     H ONCOL|
+----+--------------------+------------+
only showing top

In [57]:
step3 = step2.select('uuid', 'room_uuid', 'facility_code', 'room_code', 'term_code')

In [83]:
stepA= subject_memberships.join(subjects, subject_memberships.subject_code==subjects.code, how = 'inner').select('name', 'abbreviation', 'course_offering_uuid')
final = stepA.join(step3, step3.uuid==stepA.course_offering_uuid, how='left').select('name','facility_code', 'room_code', 'term_code')


In [85]:
final.groupBy('facility_code', 'name').count().orderBy('facility_code').show()

+-------------+--------------------+-----+
|facility_code|                name|count|
+-------------+--------------------+-----+
|       ONLINE|Civil and Environ...|   18|
|       ONLINE|              German|    6|
|       ONLINE|Engineering Mecha...|    6|
|       ONLINE|Curriculum and In...|   24|
|       ONLINE|      Jewish Studies|    6|
|       ONLINE|African Languages...|    8|
|       ONLINE|Literature in Tra...|    8|
|       ONLINE|    Consumer Science|   53|
|       ONLINE|Collaborative Nur...|  117|
|       ONLINE|        Horticulture|    2|
|       ONLINE|               Dance|    4|
|       ONLINE| Nuclear Engineering|    1|
|       ONLINE|             Nursing|   38|
|       ONLINE|Interdisciplinary...|   12|
|       ONLINE|Engineering Profe...|  153|
|       ONLINE|           Sociology|    4|
|       ONLINE|            Agronomy|    4|
|       ONLINE|    Folklore Program|   11|
|       ONLINE|       Legal Studies|    3|
|       ONLINE|Languages and Cul...|    4|
+----------

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [ ]:
course_offerings: uuid, course_uuid, term_code, name

instructors: id, name

schedules: uuid

sections: uuid, course_offering_uuid,room_uuid, schedule_uuid

teachings: instructor_id, section_uuid

courses: uuid

grade_distributions: course_offering_uuid,section_number

rooms: uuid, facility_code, room_code

subjects: code

subject_memberships: subject_code, course_offering_uuid

In [63]:
agg= grade_distributions.groupBy('course_offering_uuid').sum()

In [67]:
hard = agg.join(course_offerings, agg.course_offering_uuid==course_offerings.uuid, how= 'left').select('sum(f_count)', 'name')

In [68]:
hard.orderBy('sum(f_count)', ascending=False).limit(5).toPandas()

,sum(f_count),name
0,72,Calc--Functns of Variables
1,70,Animal Biology
2,67,Calculus&Analytic Geometry
3,64,Calculus&Analytic Geometry 1
4,63,Calculus&Analytic Geometry


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [69]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [70]:
from pyspark.sql.functions import levenshtein

In [86]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

options =spark.createDataFrame(['Monkey', 'Elephant', 'Hippopotamus'],StringType())

In [88]:
results= zoo.join(options, levenshtein(zoo['Animal'], options['value']) < 5, 'left')
results.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+

